# Demo of the Two Population Model

**Note 1**: the model is still under development. Results should not yet be trusted, please discuss them with me first to see what possible problems there might be. Also note that the [Binder](http://mybinder.org/) environment is temorary and requires a internet connection. Your work will definitely be lost unless you either download it, or even better, you clone the [github repository](https://github.com/birnstiel/twopoppy-demo) on your machine and run it there.

**Note 2**: here we call the script externally and then read in the data that has been written to disk, just for demonstration purposes. The code can also be called directly from python which avoids any I/O.

-------

Execute this to download necessary files.

In [ ]:
!make SHELL=/bin/bash

General imports and setup

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='svg'
from aux import year, AU
from const import M_sun
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import style
style.use(['seaborn-dark',{'axes.grid': True,'font.size':10}]);

Run **one** of the two cells below, the first one is a solar mass star, the second one is for a brown dwarf disk.

In [ ]:
%run two_pop_run.py -rstar 1 -mdisk 1e-1 -rc 300 -tstar 4000 -mstar 1 -tmax 2e6 -alpha 1e-2

In [ ]:
%run two_pop_run.py -rstar 0.8299 -mdisk 1e-2 -rc 20 -tstar 3000 -mstar 0.1 -tmax 5e5

Reading in the data.

In [ ]:
x      = np.loadtxt('data/x.dat')
t      = np.loadtxt('data/time.dat')
sig_d  = np.loadtxt('data/sigma_d.dat')
sig_da = np.loadtxt('data/sigma_d_a.dat')
sig_g  = np.loadtxt('data/sigma_g.dat')
a      = np.loadtxt('data/a.dat')

# Plots for Giovanni

In [ ]:
m_d = np.trapz(2*np.pi*x*sig_d,x=x)
m_g = np.trapz(2*np.pi*x*sig_g,x=x)
f,ax = plt.subplots(2,1,figsize=(6,6))
ax[0].loglog(t/year,m_d/m_g,label='dust-to-gas')
ax[0].set_xlabel('time [yr]')
ax[0].set_ylabel('global dust-to-gas ratio')
ax[0].set_ylim([2e-4,2e-2])

ax[1].loglog(t/year,m_g/M_sun,label='gas')
ax[1].loglog(t/year,100*m_d/M_sun,label='dust$\\times 100$')
ax[1].set_xlabel('time [yr]')
ax[1].set_ylabel('Mass [$M_\odot$]')
ax[1].legend(loc='best')
ax[1].set_ylim([2e-5,2e-1])

ax[0].set_title('Evolution of the global dust-to-gas ratio and dust/gas masses');

In [ ]:
f,ax=plt.subplots()
ax.loglog(t[1:]/year,-np.diff(m_g)/np.diff(t)/M_sun*year)
ax.set_xlabel('time [yr]')
ax.set_ylabel('$\dot M_\mathrm{g}$ [$M_\odot$/yr]')
ax.set_title('Evolution of the gas accretion rate');

# Plots for Leonardo

In [ ]:
f,ax=plt.subplots(figsize=(6,4))
for _t in [0.1,0.2,0.5]:
    it = abs(t-_t*1e6*year).argmin()
    l,=ax.loglog(x/AU,sig_g[it]/100.,label='{} Myr'.format(_t))
    ax.loglog(x/AU,sig_d[it],c=l.get_color(),ls='--')
ax.set_xlabel('r [AU]')
ax.set_ylabel('$\Sigma$ [g cm$^{-2}$]')
ax.set_xlim(1e0,5e2)
ax.set_ylim(1e-7,1e0)

ax.plot(1e-100,1e-100,'k-',label='$\Sigma_\mathrm{g}$/100')
ax.plot(1e-100,1e-100,'k--',label='$\Sigma_\mathrm{d}$')
ax.legend(fontsize='x-small')
f.savefig('1.pdf');

In [ ]:
a0=0.01
a1=0.1

ia0 = abs(a-a0).argmin()
ia1 = abs(a-a1).argmin()

f,ax=plt.subplots(figsize=(6,4))
ax.loglog(x/AU,sig_g[it]/100.,label='$\Sigma_\mathrm{g}$/100')
ax.loglog(x/AU,sig_da[ia0:ia1+1,:].sum(0),label='$\Sigma_\mathrm{{d}}$ [{:} mm -{:} mm]'.format(10*a0,10*a1))
ax.set_xlim(1e0,5e2)
ax.set_ylim(1e-7,1e0)

ax.set_xlabel('r [AU]')
ax.set_ylabel('$\Sigma$ [g cm$^{-2}$]')
ax.legend(fontsize='small');
f.savefig('2.pdf')

Run three time to get three size distributions (this can be done much more effectively by calling the distribution reconstruction directly, but I was to lazy to do that).

In [ ]:
a0=0.01
a1=0.1

ia0 = np.abs(a-a0).argmin()
ia1 = np.abs(a-a1).argmin()

f,ax=plt.subplots(figsize=(8,6))
for _t in [0.1,0.2,0.5]:
    r      = os.system('./two_pop_run.py -rstar 0.8299 -mdisk 1e-2 -rc 20 -tstar 3000 -mstar 0.1 -tmax {}'.format(_t*1e6))
    x      = np.loadtxt('data/x.dat')
    t      = np.loadtxt('data/time.dat')
    sig_d  = np.loadtxt('data/sigma_d.dat')
    sig_da = np.loadtxt('data/sigma_d_a.dat')
    sig_g  = np.loadtxt('data/sigma_g.dat')
    a      = np.loadtxt('data/a.dat')
    
    l,=ax.loglog(x/AU,sig_g[-1]/100.,label='{} Myr'.format(_t))
    ax.loglog(x/AU,sig_da[ia0:ia1+1,:].sum(0),ls='--',c=l.get_color())
ax.set_xlim(1e0,5e2)
ax.set_ylim(1e-7,1e0)

ax.set_xlabel('r [AU]')
ax.set_ylabel('$\Sigma$ [g cm$^{-2}$]')
ax.set_xlim(1e0,5e2)
ax.set_ylim(1e-7,1e0)

ax.plot(1e-100,1e-100,'k-',label='$\Sigma_\mathrm{g}$/100')
ax.plot(1e-100,1e-100,'k--',label='$\Sigma_\mathrm{{d}}$ [{:} mm -{:} mm]'.format(10*a0,10*a1))
ax.legend(fontsize='small')
f.savefig('3.pdf');

To present the results as slides, save the notebook with the figures and then execute the cell below. Make sure to interrupt the kernel (via the "stop" button or the menu) when you are done.

In [ ]:
!jupyter nbconvert ALMA_BD_edge_plot.ipynb --to slides --post serve